In [2]:
import pickle

# feature embedding dictionary
# title, keywords, description 
page_feautre = pickle.load(open('./data/final_feature_1684.pickle','rb'))

# load the complete url-features dictionary of pages (pre-constructed)
page_allintents_title = pickle.load(open('./data/infowave_allintents_title.pkl','rb'))

# 321 pages that have been curated, dictionary of URL-id
page_id = pickle.load(open('./data/infowave_allintents_dict.pkl','rb'))

# load the complete text-features dictionary of queries (pre-constructed)
question_features = pickle.load(open('./data/all_questions_title.pkl','rb'))

# load the complete association matrix of query-page (pre-constructed)
association_mat = pickle.load(open('./data/all_question_article_ratings.pkl','rb'))

# load the complete page level scrapped text
page_text = pickle.load(open('./data/final_text_infowave_full.p','rb'))

In [3]:
import pandas as pd 
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer 
import numpy as np 
      
page_title_embeddings=np.zeros((321,768))
page_keywords_embeddings=np.zeros((321,768))
page_description_embeddings=np.zeros((321,768))


page_title_bow = np.zeros((321,400))
page_keywords_bow = np.zeros((321,400))
page_description_bow = np.zeros((321,400))

corpus_title=[]
corpus_keywords=[]
corpus_description=[]
corpus_text=[]
corpus_question=[]

for k,v in question_features.items():
    corpus_question.append(k)

for k,v in page_id.items():
    
    corpus_title.append(page_text[k]['title'])
    corpus_keywords.append(page_text[k]['keywords'])
    corpus_description.append(page_text[k]['description'])
    corpus_text.append(' '.join(page_text[k]['text']))
    
    if page_feautre[k]['title'].shape[0]>1:
        page_title_embeddings[v[0],:]=np.mean(page_feautre[k]['title'])
    else:
        page_title_embeddings[v[0],:]=page_feautre[k]['title']

    if page_feautre[k]['keywords'].shape[0]>1:
        page_keywords_embeddings[v[0],:]=np.mean(page_feautre[k]['keywords'])
    else:
        page_keywords_embeddings[v[0],:]=page_feautre[k]['keywords']
        
    if page_feautre[k]['description'].shape[0]>1:
        page_description_embeddings[v[0],:]=np.mean(page_feautre[k]['description'])
    else:
        page_description_embeddings[v[0],:]=page_feautre[k]['description']          
    


In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

#instantiate CountVectorizer()
cv=CountVectorizer(min_df=0.1, max_df=0.7, max_features=500, analyzer= 'word', stop_words='english') 
tfidfvectorizer = TfidfVectorizer(max_features=500, analyzer='word',stop_words= 'english')
 
# this steps generates word counts for the words in your docs 
page_count_text_vector=cv.fit_transform(corpus_text)
page_tfidf_text_vector=tfidfvectorizer.fit_transform(corpus_text)

page_count_title_vector=cv.fit_transform(corpus_title)
page_tfidf_title_vector=tfidfvectorizer.fit_transform(corpus_title)

page_count_keywords_vector=cv.fit_transform(corpus_keywords)
page_tfidf_keywords_vector=tfidfvectorizer.fit_transform(corpus_keywords)

page_count_description_vector=cv.fit_transform(corpus_description)
page_tfidf_description_vector=tfidfvectorizer.fit_transform(corpus_description)

question_count_vector=cv.fit_transform(corpus_question)
question_tfidf_vector=tfidfvectorizer.fit_transform(corpus_question)

# count_wm = countvectorizer.fit_transform(train)
# tfidf_wm = tfidfvectorizer.fit_transform(train)


# #count_vector=cv.transform(corpus_text) 

# tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
# tfidf_transformer.fit(word_count_vector)

# tf_idf_vector=TfidfTransformer.transform(count_vector)

# tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
# df_idf=tfidf_transformer.fit(word_count_vector)

# # print idf values 
# df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])

# # sort ascending 
# df_idf.shape

In [42]:
# reconstruct the dictionary for question features
question_features_dict={}

ipos = 0
for k,v in question_features.items():
    question_features_dict[k]=question_tfidf_vector[ipos,:].todense()
    ipos+=1


In [47]:
# save the page embeddings

# pickle.dump(page_title_embeddings, open('./data/X_page_title_embeddings.pkl','wb'))
# pickle.dump(page_keywords_embeddings, open('./data/X_page_keywords_embeddings.pkl','wb'))
# pickle.dump(page_description_embeddings, open('./data/X_page_description_embeddings.pkl','wb'))

pickle.dump(page_tfidf_text_vector.todense(), open('./data/X_page_text_tfidf.pkl','wb'))
pickle.dump(page_tfidf_title_vector.todense(), open('./data/X_page_title_tfidf.pkl','wb'))
pickle.dump(page_tfidf_keywords_vector.todense(), open('./data/X_page_keywords_tfidf.pkl','wb'))
pickle.dump(page_tfidf_description_vector.todense(), open('./data/X_page_description_tfidf.pkl','wb'))
#pickle.dump(question_tfidf_vector.todense(), open('./data/X_question_tfidf.pkl','wb'))


In [44]:
import pandas as pd
import numpy as np
data=pd.read_csv('./data/data_label.tsv',sep='\t')

query_id=0

ass_mat = np.zeros((986,321))

query_embeddings=np.zeros((986,768))
query_tfidf =np.zeros((986,500))

for index, row in data.iterrows():
    
    if (row['BestInfoWave'] in page_id) or (row['OtherInfoWave'] in page_id):
        print(row['CallerQuestion'])
        print(len(question_features[row['CallerQuestion']]))
        if (row['BestInfoWave'] in page_id):
            ass_mat[query_id,page_id[row['BestInfoWave']]]=1
            print(len(page_id[row['BestInfoWave']]))
        elif (row['OtherInfoWave'] in page_id):
            print(len(page_id[row['OtherInfoWave']]))
            ass_mat[query_id,page_id[row['OtherInfoWave']]]=1
        query_embeddings[query_id,:]=question_features[row['CallerQuestion']]
        query_tfidf[query_id,:]=question_features_dict[row['CallerQuestion']]
        query_id+=1
    
print(query_id)            
    

3 Rollovers, 1 was Roth but went into a pretax account. How to fix?
768
1
can client fax/msg rollover certificaiton letter?
768
1
Can client have a rollover wired into Vanguard?
768
1
Can client move money from ESA to another type of account?
768
1
Can client return Indirect rollover in increments?
768
1
can rollover go into any acct?
768
1
can send OATS with the rollover check?
768
1
Can we accept a direct rollover check without an account number on it?
768
1
can we fax in rollover cert letter
768
1
can we fill out the tsp 60 form?
768
1
can we rollover a 401k in kind
768
1
Can we take a rollover from a lawsuit settlement in an employer plan?
768
1
can you do a rollover to an IRA from an IRA?
768
1
client did an outgoing rollover and said there was a from we were supposed to sign and we didnt do it
768
1
Client is staating she sent us a rollover check that should have arrived early march, shouldn't I at least see record in AWD?
768
1
Client's TA IRA doesn't have VFTC in the registrati

In [45]:
pickle.dump(query_embeddings,open('./data/X_query_embeddings.pkl','wb'))
pickle.dump(query_tfidf,open('./data/X_query_tfidf.pkl','wb'))
pickle.dump(ass_mat,open('./data/X_ass_mat.pkl','wb'))

In [21]:
data

,CallerQuestion,SubCategory,BestInfoWave,OtherInfoWave,Answer,valid
0,"3 Rollovers, 1 was Roth but went into a pretax...",Direct/Indirect Rollover,http://oedesktop.vanguard.com/us/content/InfoW...,NaN,IRA contribution updates (IRA Contribution typ...,True
1,can client fax/msg rollover certificaiton letter?,Direct/Indirect Rollover,http://oedesktop.vanguard.com/us/content/InfoW...,NaN,We can receive items or requests from clients ...,True
2,Can client have a rollover wired into Vanguard?,Direct/Indirect Rollover,http://oedesktop.vanguard.com/us/content/InfoW...,NaN,"Remember: When wiring into an IRA, the client...",True
3,Can client move money from ESA to another type...,Direct/Indirect Rollover,http://oedesktop.vanguard.com/us/content/InfoW...,NaN,A movement from an ESA into a 529 plan is proc...,True
4,Can client return Indirect rollover in increme...,Direct/Indirect Rollover,http://oedesktop.vanguard.com/us/content/InfoW...,NaN,Clients may return all or a part of the single...,True
5,can rollover go into any acct?,Direct/Indirect Rollover,http://oedesktop.vanguard.com/us/content/InfoW...,http://oedesktop.vanguard.com/us/content/InfoW...,NaN,True
6,can send OATS with the rollover check?,Direct/Indirect Rollover,http://oedesktop.vanguard.com/us/content/InfoW...,NaN,Acceptable use of OATS form: Check displays an...,True
7,Can we accept a direct rollover check without ...,Direct/Indirect Rollover,http://oedesktop.vanguard.com/us/content/InfoW...,NaN,NaN,True
8,can we fax in rollover cert letter,Direct/Indirect Rollover,http://oedesktop.vanguard.com/us/content/InfoW...,NaN,NaN,True
9,can we fill out the tsp 60 form?,Direct/Indirect Rollover,http://oedesktop.vanguard.com/us/content/InfoW...,NaN,Rollover Certification Forms (Excel Spreadsheet),True


In [16]:
import random 
random.sample
sample_list = random.sample(range(995),300)

def sample_by_query_id(sample_list):
    # sample the data by query-id, 
    # sample_threhold = 900
    
    ratings = []
    
    #dict_i_mapping={}  # for hold out validation setting, we should not shrink the page id
    
    dict_q_mapping={}  

    # construct the dictionaries 
    dict_url={}
    dict_title_embed={}  
    
    dict_question={}
    dict_question_embed={}
    dict_question_intents={}
    
    #filter out query-id and page-id by association matrix
    for x,y in association_mat:
        #if (x<sample_threhold): # filter by query id
        if (x in sample_list): # filter by query id
            ratings.append((x,y))
            
    all_q_ids = [x for (x,y) in ratings]
    #all_i_ids = [y for (x,y) in ratings]            
    
    #print(all_i_ids)
    
    icount = 0
    for k,v in question_id.items():
        if (v in all_q_ids) and (k in question_features):
            dict_question[k]=v
            dict_question_embed[k]=question_features[k]
            if v not in dict_q_mapping:
                dict_q_mapping[v]=icount
                icount+=1    

    
    icount = 0

    for k,v in page_id.items():
        #print(k,v)
        #if ((v[0] in all_i_ids) and (k in page_features)):
        if k in page_features:                
            #print('appending page id to dictionary')
            dict_url[k]=v[0]
            dict_title_embed[k]=page_features[k]
            
#             if v[0] not in dict_i_mapping:
#                 dict_i_mapping[v[0]]=icount
#                 icount+=1
                
#   print(dict_q_mapping)
   
    
    new_ratings=[]

    # remap the page id to the sampled new ids 
    for x,y in ratings:
        new_ratings.append((dict_q_mapping[x],y))
    
    
    # replace the rating association matrix with the one with new ID-mappings
    ratings = new_ratings

    # construct the dictionaries 
    dict_url={}
    dict_url_embed={}    

    dict_question={}
    dict_question_embed={}
    #dict_q_mapping={}

    for k,v in page_id.items():
        #if (v[0] in all_i_ids) and (k in page_features):
        if k in page_features:

            dict_url[k]=v[0]
            dict_url_embed[k]=page_features[k]    
            
    for q_id in all_q_ids:
        #print(dict_q_mapping[q_id])
        #print(q_id)
        dict_question_intents[dict_q_mapping[q_id]]=question_intent_features[q_id]            
    
    #print(dict_i_mapping)
    dict_q_mapping={}
    icount =0
    for k,v in question_id.items():
        if (v in all_q_ids) and (k in question_features):
            dict_question[k]=icount
            dict_question_embed[k]=question_features[k]
            
            if v not in dict_q_mapping:
                dict_q_mapping[v]=icount
                icount+=1
                
    pickle.dump(dict_question,open('data/sample_by_question_questions_dict.pkl','wb'))
    pickle.dump(dict_question_embed,open('data/sample_by_question_questions_features.pkl','wb'))            
    pickle.dump(ratings,open('data/sample_by_question_questions_article_ratings.pkl','wb'))
    pickle.dump(dict_url,open('data/sample_by_question_page_dict.pkl','wb'))
    pickle.dump(dict_title_embed,open('data/sample_by_question_page_features.pkl','wb'))
    pickle.dump(dict_question_intents,open('data/sample_by_question_query_intents.pkl','wb'))

sample_by_query_id(sample_list) 

[(0, 0),
 (1, 1),
 (2, 2),
 (3, 3),
 (4, 4),
 (5, 5),
 (6, 6),
 (7, 7),
 (8, 8),
 (9, 8),
 (10, 9),
 (11, 10),
 (12, 11),
 (13, 8),
 (14, 12),
 (15, 13),
 (16, 6),
 (17, 14),
 (18, 15),
 (19, 14),
 (20, 6),
 (21, 16),
 (22, 13),
 (23, 13),
 (24, 17),
 (25, 13),
 (26, 17),
 (27, 15),
 (28, 18),
 (29, 14),
 (30, 19),
 (31, 20),
 (32, 21),
 (33, 20),
 (34, 22),
 (35, 8),
 (36, 0),
 (37, 23),
 (38, 8),
 (39, 24),
 (40, 8),
 (41, 25),
 (42, 26),
 (43, 25),
 (44, 25),
 (45, 25),
 (46, 25),
 (47, 25),
 (48, 25),
 (49, 27),
 (50, 25),
 (51, 25),
 (52, 28),
 (53, 25),
 (54, 0),
 (55, 29),
 (56, 30),
 (57, 25),
 (58, 31),
 (59, 31),
 (60, 32),
 (61, 31),
 (62, 25),
 (63, 30),
 (64, 33),
 (65, 26),
 (67, 34),
 (68, 26),
 (69, 35),
 (70, 36),
 (71, 25),
 (72, 26),
 (73, 34),
 (74, 37),
 (75, 34),
 (76, 34),
 (77, 38),
 (78, 29),
 (79, 26),
 (80, 39),
 (81, 40),
 (82, 41),
 (83, 42),
 (84, 42),
 (85, 42),
 (86, 43),
 (87, 42),
 (88, 44),
 (89, 45),
 (90, 46),
 (91, 42),
 (92, 47),
 (93, 42),
 (94, 

In [5]:
icount = 0

for k,v in page_feautre.items():
    if icount<2:
        print(k,v)
    icount+=1

http://oedesktop.vanguard.com/us/content/InfoWave/AssocHbk/ContactVG/BusinessCardsFlagship.jsp {'title': array([[ 1.79043934e-01, -6.10523149e-02, -8.49802494e-02,
         2.60922283e-01,  1.80476189e-01,  1.19623385e-01,
        -1.40463347e-02, -1.60392255e-01, -2.12920040e-01,
         6.35203496e-02, -6.37770537e-03, -2.20238995e-02,
         1.90845542e-02,  1.86672375e-01, -1.95051879e-01,
        -2.19055742e-01, -6.14605904e-01,  3.58545214e-01,
        -1.28804862e-01, -9.80308503e-02, -1.93770245e-01,
         1.38204563e-02,  5.36382616e-01,  1.37386784e-01,
         3.56579781e-01,  3.10587455e-02, -1.52456999e-01,
         8.62757787e-02, -3.34871441e-01,  4.82046492e-02,
         1.32270947e-01,  6.70087561e-02,  2.94868439e-01,
         3.59520949e-02, -2.20440269e-01, -5.28046787e-01,
        -7.34810298e-03, -2.69279927e-01,  4.92234752e-02,
        -1.53400898e-01, -4.72148210e-01, -1.70310900e-01,
         1.91770926e-01, -8.98679346e-02,  7.57736489e-02,
         2